In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
from logging import INFO, StreamHandler, getLogger

logger = getLogger()
log_handler = StreamHandler(sys.stdout)
logger.addHandler(log_handler)
logger.setLevel(INFO)

# Import libraries

In [ ]:
import os
from glob import glob
from logging import INFO, StreamHandler, getLogger

import matplotlib.pyplot as plt
import numpy as np
import torch
from IPython.display import HTML, display
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

ROOT_DIR = "/workspace"

sys.path.append(f"{ROOT_DIR}/pytorch/src")
from io_fortran_data import read_vortex_file

# Define constants

In [ ]:
TRAIN_DATA_DIR = f"{ROOT_DIR}/data/fortran/decaying_turbulence/train"
TEST_DATA_DIR = f"{ROOT_DIR}/data/fortran/decaying_turbulence/test"
OUTPUT_DIR = f"{ROOT_DIR}/data/pytorch/decaying_turbulence/DL_data/vorticity"

TSTEP_START = 110  # inclusive
TSTEP_END = 200  # inclusive
TSTEP_INTERVAL = 10
DT = 0.01

K_MAX = 42
NX = 128
NY = NX

# Check data

In [ ]:
xs = np.linspace(0, 2 * np.pi, NX)
ys = np.linspace(0, 2 * np.pi, NY)
X, Y = np.meshgrid(xs, ys, indexing="ij")

n_cols = 5
n_rows = 2

for kind, dir_path in zip(["train", "test"], [TRAIN_DATA_DIR, TEST_DATA_DIR]):
    display(HTML(f"<h2>{kind}</h2>"))
    for file_path in glob(f"{dir_path}/T{K_MAX}*.dat")[:2]:
        vortex_field = read_vortex_file(file_path, NX, NY)

        fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True, figsize=[10, 4])

        for it, (Z, ax) in enumerate(
            zip(
                vortex_field[TSTEP_START : TSTEP_END + TSTEP_INTERVAL : TSTEP_INTERVAL, :, :],
                np.ravel(axes),
            )
        ):
            vmin, vmax = np.min(Z), np.max(Z)
            vmin, vmax = -max([np.abs(vmin), np.abs(vmax)]), max([np.abs(vmin), np.abs(vmax)])
            levels = np.linspace(vmin, vmax, 21)

            ax.axis("off")
            ax.contourf(X, Y, Z, cmap="coolwarm", levels=levels)
            t = DT * (TSTEP_START + it * TSTEP_INTERVAL)
            ax.set_title(f"t = {t:.2f}\nabs max = {vmax:.0f}")
            ax.set_aspect("equal")

        plt.tight_layout()
        plt.show()

# Make data for DL

## Train and validation data

In [ ]:
all_file_paths = sorted(glob(f"{TRAIN_DATA_DIR}/T{K_MAX}*.dat"))
print(f"Total file num = {len(all_file_paths)}")

train_file_paths, valid_file_paths = train_test_split(all_file_paths, test_size=0.3, shuffle=False)

for kind, paths in zip(["train", "valid"], [train_file_paths, valid_file_paths]):
    print(f"kind = {kind}, len(paths) = {len(paths)}")
    os.makedirs(f"{OUTPUT_DIR}/{kind}", exist_ok=False)

    for path in tqdm(paths):
        Zs = read_vortex_file(path, NX, NY)
        for it in range(TSTEP_START, TSTEP_END + TSTEP_INTERVAL, TSTEP_INTERVAL):
            # e.g., T85_seed12382771132014.dat --> seed12382771132014
            seed_info = os.path.basename(path).split("_")[-1].split(".")[0]
            output_path = f"{OUTPUT_DIR}/{kind}/T{K_MAX}_{seed_info}_it{it:04}.npy"
            np.save(output_path, Zs[it, :, :])

## Test data

In [ ]:
kind = "test"
paths = sorted(glob(f"{TEST_DATA_DIR}/T{K_MAX}*.dat"))

print(f"kind = {kind}, len(paths) = {len(paths)}")
os.makedirs(f"{OUTPUT_DIR}/{kind}", exist_ok=False)

for path in tqdm(paths):
    Zs = read_vortex_file(path, NX, NY)
    for it in range(TSTEP_START, TSTEP_END + TSTEP_INTERVAL, TSTEP_INTERVAL):
        # e.g., T85_seed12382771132014.dat --> seed12382771132014
        seed_info = os.path.basename(path).split("_")[-1].split(".")[0]
        output_path = f"{OUTPUT_DIR}/{kind}/T{K_MAX}_{seed_info}_it{it:04}.npy"
        np.save(output_path, Zs[it, :, :])

# Check train, valid, and test data

In [ ]:
xs = np.linspace(0, 2 * np.pi, NX)
ys = np.linspace(0, 2 * np.pi, NY)
X, Y = np.meshgrid(xs, ys, indexing="ij")

for kind in ["train", "valid", "test"]:
    display(HTML(f"<h2>{kind}</h2>"))

    dir_path = f"{OUTPUT_DIR}/{kind}"
    first_path = sorted(glob(f"{dir_path}/*.npy"))[0]
    seed_info = os.path.basename(first_path).split("_")[1]

    assert len(glob(f"{dir_path}/*{seed_info}*")) == (TSTEP_END - TSTEP_START) // TSTEP_INTERVAL + 1
    paths = sorted(glob(f"{dir_path}/*{seed_info}*"))[:10]

    fig, axes = plt.subplots(2, 5, sharex=True, sharey=True, figsize=[10, 4])

    for path, ax in zip(paths, np.ravel(axes)):
        Z = np.load(path)
        vmin, vmax = np.min(Z), np.max(Z)
        vmin, vmax = -max([np.abs(vmin), np.abs(vmax)]), max([np.abs(vmin), np.abs(vmax)])
        levels = np.linspace(vmin, vmax, 21)

        ax.axis("off")
        ax.contourf(X, Y, Z, cmap="coolwarm", levels=levels)
        ax.set_title(f"{os.path.basename(path).split('_')[-1].split('.')[0]}")
        ax.set_aspect("equal")

    plt.tight_layout()
    plt.show()

In [ ]:
kind = "valid"
dir_path = f"{OUTPUT_DIR}/{kind}"
paths = sorted(glob(f"{dir_path}/*.npy"))

all_zs = []
for path in paths:
    all_zs.append(np.load(path).flatten())
all_zs = np.concatenate(all_zs)
len(all_zs) // (NX * NY)

In [ ]:
std = np.std(all_zs)
p25, p75 = np.percentile(all_zs, [25, 75])
std = p75 - p25
xs = np.linspace(-200, 200, 401)
ys = 1 / np.sqrt(2 * np.pi * std ** 2) * np.exp(-0.5 * xs ** 2 / std ** 2)
print(std, np.mean(all_zs))

In [ ]:
plt.hist(all_zs, bins=400, range=(-200, 200), density=True)
plt.plot(xs, ys)
plt.show()

In [ ]:
datasets = VortexDatasetForDecayingTurbulence(
    data_dir=f"{OUTPUT_DIR}/valid",
    mean=0.0,
    std=16.0,
    scale=8,
    image_width=None,
    image_height=None,
    lr_method="subsample",
    num_simulations=15,
)

In [ ]:
all_xs, all_ys = [], []

for i, (x, y) in enumerate(iter(datasets)):
    all_xs.append(x.numpy().flatten())
    all_ys.append(y.numpy().flatten())
    if i + 1 <= 5:
        print(f"x size = {x.shape}, y size = {y.shape}")
        vmin, vmax = torch.min(y), torch.max(y)
        ax = plt.subplot(121)
        ax.imshow(x.numpy().squeeze(), vmin=vmin, vmax=vmax, interpolation=None)
        ax = plt.subplot(122)
        ax.imshow(y.numpy().squeeze(), vmin=vmin, vmax=vmax, interpolation=None)
        plt.show()
all_xs, all_ys = np.concatenate(all_xs), np.concatenate(all_ys)

In [ ]:
plt.hist(all_xs, range=(-10, 10), bins=101)
plt.show()
plt.hist(all_ys, range=(-10, 10), bins=101)
plt.show()